In [1]:
import cv2 
import os 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

# Step 1 - Write a Download Script
Write a script or function that downloads a .ts file and extracts the first frame as a jpg image.

example script

​> ​./fetch-and-extract 1538076003

downloading https://hiring.verkada.com/video/1538076003.ts...

extracting image... wrote 1538076003.jpg

In [14]:
filename = 'index.txt'
with open(filename) as f:
    data = f.readlines()
for i in range(2000,8000,60):
    timestamp = data[i][:-1]
    
    # Read the video
    cam = cv2.VideoCapture('https://hiring.verkada.com/video/'+timestamp)
    print('downloading https://hiring.verkada.com/video/'+timestamp)
    try: 
        # creating a folder named image
        if not os.path.exists('image'): 
            os.makedirs('image') 
    # if not created then raise error 
    except OSError: 
        print ('Error: Creating directory of image') 

    # reading from frame 
    ret,frame = cam.read() 

    if ret: 
        # if video is still left continue creating images 
        name = './image/' + timestamp[:-3] + '.jpg'
        print ('extracting image...wrote ' + name) 

        # writing the extracted images 
        cv2.imwrite(name, frame) 

    # Release all space and windows once done 
    cam.release() 
    cv2.destroyAllWindows() 

downloading https://hiring.verkada.com/video/1538084161.ts
extracting image...wrote ./image/1538084161.jpg
downloading https://hiring.verkada.com/video/1538084400.ts
extracting image...wrote ./image/1538084400.jpg
downloading https://hiring.verkada.com/video/1538084640.ts
extracting image...wrote ./image/1538084640.jpg
downloading https://hiring.verkada.com/video/1538084880.ts
extracting image...wrote ./image/1538084880.jpg
downloading https://hiring.verkada.com/video/1538085120.ts
extracting image...wrote ./image/1538085120.jpg
downloading https://hiring.verkada.com/video/1538085360.ts
extracting image...wrote ./image/1538085360.jpg
downloading https://hiring.verkada.com/video/1538085600.ts
extracting image...wrote ./image/1538085600.jpg
downloading https://hiring.verkada.com/video/1538085840.ts
extracting image...wrote ./image/1538085840.jpg
downloading https://hiring.verkada.com/video/1538086080.ts
extracting image...wrote ./image/1538086080.jpg
downloading https://hiring.verkada.co

downloading https://hiring.verkada.com/video/1538102636.ts
extracting image...wrote ./image/1538102636.jpg
downloading https://hiring.verkada.com/video/1538102876.ts
extracting image...wrote ./image/1538102876.jpg
downloading https://hiring.verkada.com/video/1538103116.ts
extracting image...wrote ./image/1538103116.jpg
downloading https://hiring.verkada.com/video/1538103356.ts
extracting image...wrote ./image/1538103356.jpg
downloading https://hiring.verkada.com/video/1538103597.ts
extracting image...wrote ./image/1538103597.jpg
downloading https://hiring.verkada.com/video/1538103837.ts
extracting image...wrote ./image/1538103837.jpg
downloading https://hiring.verkada.com/video/1538104077.ts
extracting image...wrote ./image/1538104077.jpg
downloading https://hiring.verkada.com/video/1538104317.ts
extracting image...wrote ./image/1538104317.jpg
downloading https://hiring.verkada.com/video/1538104557.ts
extracting image...wrote ./image/1538104557.jpg
downloading https://hiring.verkada.co

# Step 2 - Extract the car in the box shown above

We’ll only look at a single parking spot (the one shown above in the red bounding box). Write a script or function that accepts a single image and runs the yolo3 algorithm (or the algorithm of your choice) on the spot above.

example script

    >./has-car 1538076003.jpg
    
running yolo3 on 1538076003.jpg...

no car detected!

In [2]:
# Initialize the parameters
confThreshold = 0.5  #Confidence threshold
nmsThreshold = 0.4   #Non-maximum suppression threshold
inpWidth = 416       #Width of network's input image
inpHeight = 416      #Height of network's input image

# Load names of classes
classesFile = "darknet/data/coco.names"
classes = None
with open(classesFile, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')

In [3]:
# load pretrained YOLO object detector trained on COCO dataset (80 classes)
print('loading YOLO...')
net = cv2.dnn.readNetFromDarknet('darknet/cfg/yolov3.cfg', 'yolov3.weights')
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
# could try setting the preferable target to cv.dnn.DNN_TARGET_OPENCL to run it on a GPU
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU) 

loading YOLO...


In [4]:
def extract_car(image_path):
    
    image = cv2.imread(image_path)
    (H, W) = image.shape[:2]
    
    #print('running yolo3 on '+image_path[6:])
    
    # determine only the *output* layer names that we need from YOLO
    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    # construct a blob from the input image and then perform a forward
    # pass of the YOLO object detector, giving us our bounding boxes and
    # associated probabilities
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (inpWidth, inpHeight),swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(ln)
    
    boxes = []
    confidences = []
    classIDs = []
    # loop over each of the layer outputs
    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            # extract the class ID and confidence (i.e., probability) of the current object detection
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # filter out weak predictions by ensuring the detected probability is greater than the minimum probability
            if confidence > confThreshold:
                # scale the bounding box coordinates back relative to the
                # size of the image, keeping in mind that YOLO actually
                # returns the center (x, y)-coordinates of the bounding
                # box followed by the boxes' width and height
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                # use the center (x, y)-coordinates to derive the top and
                # and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                # update our list of bounding box coordinates, confidences, and class IDs
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)
    # apply non-maxima suppression to suppress weak, overlapping bounding boxes
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)

   
    # ensure at least one detection exists
    if len(idxs) > 0:
        # loop over the indexes we are keeping
        for i in idxs.flatten():
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            if x in range(180,210) and y in range(180,210) and classes[classIDs[i]] in ['car','truck']:
                # extract car from image
                car = image[y:y+h, x:x+w]
                #cv2.imwrite('./car/'+image_path[6:], car)
                #print('car detected!')
                return extract_features(car)

        #print('no car detected!')
        return False

# Step 3 - Compute the similarity between two cars

Write a script or function that accepts two images. Assume that both images contain a car. Compute whether or not the image is of the same car. The result should be a boolean.
example script

       > ./fetch-and-extract 1538076183
       > ./fetch-and-extract 1538076179
       > ./is-same-car 1538076179.jpg 1538076183.jpg
       comparing 1538076179.jpg and 1538076183.jpg... same car!

In [5]:
def extract_features(image):
    
    # Initiate SIFT detector
    sift = cv2.xfeatures2d.SIFT_create()

    # find the keypoints and descriptors with SIFT
    kp, des = sift.detectAndCompute(image, None)
    
    return (kp, des)

In [23]:
def extract_features_matching(feature, matching):
    
    if not feature or not matching:
        return False

    # find the keypoints and descriptors with SIFT
    kp1, des1 = feature[0], feature[1]
    kp2, des2 = matching[0], matching[1]

    # FLANN parameters
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)   # or pass empty dictionary

    flann = cv2.FlannBasedMatcher(index_params, search_params)

    matches = flann.knnMatch(des1, des2, k=2)
    
    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.8*n.distance:
            good.append(m)
            
    MIN_MATCH_COUNT = 3
    if len(good) > MIN_MATCH_COUNT:
        #print('... same car!')
        return True
    else:
        #print('... different car!')
        return False

# Step 4 - Put it all together!

Write a script or function that accepts a time range and outputs each car that was detected and how long it was parked for (approximately). You’ll probably want to get the index file to see which timestamps are available for download.

The output format or return value for all steps is up to you. A possible sample output is shown below for a python shell script:

       > curl https://hiring.verkada.com/video/index.txt > index.txt
       > python3 analyze-cars.py --index index.txt --start 1538076003 --end 1538078234
       
       analyzing from 1538076003 to 1538078234
       found car at 1538076175. parked until 1538077874 (28 minutes).
       ... wrote output/1538076175-28min.jpg
       found car at 1538077954. parked until 1538078198 (4 minutes).
       ... wrote output/1538077954-4min.jpg
       no more cars found!

In [25]:
filename = 'index.txt'
with open(filename) as f:
    data = f.readlines()

print('analyzing from '+ data[2000][:-4] + ' to ' + data[8000][:-4])

images_path = 'image/'
files = [os.path.join(images_path, p) for p in sorted(os.listdir(images_path))]

# initial state of observed parking spot
occupied = False
# features of the car parked in this spot, initial None 
occupied_feature = None
start = None

for i in range(1,len(files)):
    cur_car_feature = extract_car(files[i])
    if cur_car_feature:   # if there is a car found in this spot in current img
        if not occupied:  # if the spot is not occupied
            occupied = True
            occupied_feature = cur_car_feature
            start = files[i][6:-4]
            print('found car at ' + start , end = '')
        
        # if the spot is occupied, then compare these two cars to check if they are same  
        else:
            # if they are different, calculate the parking time
            if not extract_features_matching(cur_car_feature, occupied_feature):
                duration = int((int(files[i][6:-4])-int(start))/60)
                print(' parked until ' + files[i-1][6:-4] + '(%s minutes).'% duration)
                print('...wrote output/'+start+'-%smin.jpg' %duration)
                start = files[i][6:-4]
                print('found car at ' + start , end = '')
            occupied_feature = cur_car_feature
    # if there is not car found in this spot in current img
    else:
        if occupied:
            duration = int((int(files[i][6:-4])-int(start))/60)
            print(' parked until ' + files[i-1][6:-4] + '(%s minutes).'% duration)
            print('...wrote output/'+start+'-%smin.jpg' %duration)

            occupied_feature = None
            start = None
            occupied = False
            

analyzing from 1538084161 to 1538108158
found car at 1538084880 parked until 1538097117(207 minutes).
...wrote output/1538084880-207min.jpg
found car at 1538097597 parked until 1538098556(19 minutes).
...wrote output/1538097597-19min.jpg
found car at 1538098796 parked until 1538102636(68 minutes).
...wrote output/1538098796-68min.jpg
found car at 1538102876 parked until 1538104797(36 minutes).
...wrote output/1538102876-36min.jpg
found car at 1538105517 parked until 1538105997(12 minutes).
...wrote output/1538105517-12min.jpg
